<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/10_Module_2_Notebook_VI_PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ranking Graph Data

Let's look at link analysis!

## Autograder setup

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
%set_env HW_ID=cis5450_25f_HW9

In [ ]:
!pip3 install penngrader-client

In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

## Load LinkedIn Graph

In [ ]:
!wget https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

In [ ]:
import pandas as pd

people_df = pd.read_json('linkedin_anon.jsonl', lines=True, nrows=5000)

In [ ]:
# Replace the index with a numeric one
people_df['_id'] = people_df.index

In [ ]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')

people_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests']).merge(names_df,on='_id')

In [ ]:
experience_df

Now let's make some changes to convert people IDs to integers, and create IDs for the organizations they've worked for.

In [ ]:
organizations_df = experience_df['org'].drop_duplicates()

org_id_start = len(people_df)

organizations_df = organizations_df.reset_index().rename(columns={'org':'name'})
organizations_df['index'] = organizations_df['index'].apply(lambda x: x + org_id_start)
organizations_df

Here is a relation of edges now.

In [ ]:
edges_df = experience_df[['_id','org']].merge(organizations_df,left_on='org',right_on='name').\
  drop(columns=['name','org']).rename(columns={'_id': 'from', 'index': 'to'})

edges_reversed_df = edges_df.rename(columns={'from': 'to', 'to': 'from'})

edges_df = pd.concat([edges_df, edges_reversed_df]).drop_duplicates()
edges_df

In [ ]:
max(edges_df['from'])

In [ ]:
max(edges_df['to'])

For PageRank we need to scale the weight transfer by the number of out-edges. To do that let's count the number of out-links for each node.  This gets all of the edges, except for nodes that have no connections!

In [ ]:
outlinks_df = edges_df.groupby('from').size().to_frame('outlinks')
display(outlinks_df)

Here are the organizations and people without edges.  We'll use this result but it won't actually affect the computation...  Nonetheless it's good to understand how to compute it.  You'll see that there *are* people who have no edges.

In [ ]:
# This is equivalent to a "left antijoin": organizations where there is a left_only result, i.e., no match on the right
sinks_df = organizations_df.merge(edges_df, how='left', left_on='index', right_on='from', indicator=True).query("_merge == 'left_only'")[['index']].rename(columns={'index': 'from'})
# Similar to above but in reverse.
sinks_df_2 = people_df.merge(edges_df, how='left', left_on='_id', right_on='from', indicator=True).query("_merge == 'left_only'")[['_id']].rename(columns={'_id': 'from'})
sinks_df = pd.concat([sinks_df, sinks_df_2])
sinks_df['outlinks'] = 0
sinks_df = sinks_df.set_index('from')
sinks_df

In [ ]:
outlinks_df = pd.concat([outlinks_df, sinks_df])
outlinks_df

Now we can see the whole process.  We'll join the edges with their weight transfers. Note that this means nodes with no weight transfer won't be considered.

In [ ]:
weight_transfer_df = edges_df.merge(outlinks_df, left_on='from', right_index=True)
weight_transfer_df['weight'] = weight_transfer_df['outlinks'].apply(lambda x: 1 / x if x > 0 else 0)
weight_transfer_df

In [ ]:
weight_transfer_df = weight_transfer_df.drop(columns=['outlinks']).dropna()
weight_transfer_df

## Convert to a Matrix

Now let's convert to a matrix...

In [ ]:
len(weight_transfer_df)


In [ ]:
import numpy as np

def compute_matrix(node_df, weight_transfer_df):
  """
  node_df: set of nodes in a dataframe, with unique node index values
  weight_transfer_df: set of edges in a dataframe, with weights
  """
  max_id = max(organizations_df['index'])

  weight_transfer_mat = np.zeros((max_id+1, max_id+1))

  for i, row in weight_transfer_df.iterrows():
    weight_transfer_mat[int(row['to']), int(row['from'])] = row['weight']

  return weight_transfer_mat

def matrix_pagerank(weight_transfer_mat, max_iter=100, alpha=0.85):
  """
  weight_transfer_mat: matrix of weights
  alpha: damping factor
  max_iter: maximum number of iterations
  """
  beta = 1 - alpha
  n = weight_transfer_mat.shape[0]
  # Initialize PageRank to 1/n
  pr = np.ones(weight_transfer_mat.shape[0]) / weight_transfer_mat.shape[0]

  for i in range(max_iter):
    pr = beta * (weight_transfer_mat @ pr) + alpha / weight_transfer_mat.shape[0]

  return pr

In [ ]:
weight_transfer_mat = compute_matrix(organizations_df, weight_transfer_df)

pr = matrix_pagerank(weight_transfer_mat, 5)
pr

## Exercise

Suppose that we have people who are only connected to a single organization, organizations with a single person. Given that we are treating edges as bidirectional: such nodes, even though they act very much like a *sink*, technically have one out-edge (back to whoever connects to them).

Can you remove all *self-loops* and *sinks* from the graph, and recompute the matrices?

In [ ]:
# TODO:
edges_df_2 = # TODO

len(edges_df_2)

In [ ]:
outlinks_df_2 = # TODO

weight_transfer_df_2 = # TODO
weight_transfer_df_2

In [ ]:
weight_transfer_mat_2 = compute_matrix(organizations_df, weight_transfer_df_2)

pr2 = matrix_pagerank(weight_transfer_mat_2, 5)
pr2

In [ ]:
grader.grade('pagerank', pr2[0:1000])